
# Librerie

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import poppy
import astropy.units as u
import logging
logging.basicConfig(level=logging.DEBUG)
from astropy.io import fits
from astropy.modeling import models, fitting
import tifffile
import numpy as np
import matplotlib.colors as mcolors
from astropy.stats import sigma_clip
logging.getLogger().setLevel(logging.INFO)
from scipy.optimize import curve_fit
import photutils
import time
import datetime as dt
from scipy import fftpack
import nbformat
import plotly.graph_objects as go
from PIL import Image
from turbustat.statistics import PowerSpectrum
from scipy.signal import butter, filtfilt
import ipywidgets as widgets
from IPython.display import display, clear_output
from astropy.io.fits import Header
from scipy.signal import find_peaks
import os
import csv

# Colormap
dark_yellow = [ '#003f5c', '#2f4b7c', '#665191', '#a05195', '#d45087', '#f95d6a', '#ff7c43', '#ffa600']
custom_cmap = mcolors.LinearSegmentedColormap.from_list("DarkYellow", dark_yellow)
gradient = np.linspace(0, 1, 256).reshape(1, -1)



<span style="color:yellow; font-size:40px;">Variabili in input</span> 


In [ ]:
#############################################################
ordine = 5 #  NON OLTRE 6
imagenumber = 300 #500
stacked = False  # Fa PSD di più immagini sommate (3 al momento)
lowcut_pix = 14#15#7  #max dimension in pixel
highcut_pix = 4#9#2.1  #min dimension in pixel
lowcut= 1/lowcut_pix 
highcut = 1/highcut_pix
crop_size = 2000   # FORMATO IMMAGINE FINALE: N pixel X N pixel (dimiuisce la dimensione dell'immagine per velocizzare i calcoli)
frequency_range = 0.15    # range per trovare il picco delle spckle (centrato nella frequenza delle speckle teorica)
########################################################
speckle_threshold=7000 #1100 #7000 #9500 #8000 #9000 #6500 #9000
# Plate scale arcseconds/pixels
plate_scale = 0.0109 # 0.012#4-MARZO #0.0109 #FEBBRAIO# 0.0377 #0.0244*2 #0.0376*2 #0.0268 marzo #0.0244 febb #0.0376 #
# Diffraction limit of the telescope
aperture = 1.82                                                     # Aperture of the telescope (in meters)
wavelength = 633e-9 #700e-9                                                 # Reference wavelenght (in meters) 
diffraction_limit_radians = 2*1.22 * wavelength / aperture          # Diameter of the Airy disk
diffraction_limit_arcseconds = diffraction_limit_radians * 206265
expected_speckle_size_radians = 0.8038 *(1.22 * wavelength / aperture)
expected_speckle_size = expected_speckle_size_radians * 206265  
expected_speckle_size_pixels = expected_speckle_size / plate_scale
print("Diffraction limit in arcseconds and pixels: ",diffraction_limit_arcseconds, diffraction_limit_arcseconds/plate_scale)
print("Expected speckle size in arcseconds and pixels: ", expected_speckle_size, expected_speckle_size_pixels)

## <span style="color:green; font-size: 1em;"> Lettura file

In [ ]:

#file = (r"C:\Users\buonc\OneDrive - Alma Mater Studiorum Università di Bologna\Documents\Astrophysics and Cosmology\TESI\dati\Algol_1kHz_1_MMStack_Pos0.ome.tif")
file = (r"C:\Users\buonc\OneDrive - Alma Mater Studiorum Università di Bologna\Documents\Astrophysics and Cosmology\TESI\dati\febbraio\arcturus_633nm_10ms.ome.tif")
#file = (r"C:\Users\buonc\OneDrive - Alma Mater Studiorum Università di Bologna\Documents\Astrophysics and Cosmology\TESI\dati\marzo\4-03\Castor.tif")
#file =(r"C:\Users\buonc\OneDrive - Alma Mater Studiorum Università di Bologna\Documents\Astrophysics and Cosmology\TESI\dati\marzo\4-03\aldebaran_40ms_elev30.tif")
immagini = tifffile.imread(file)[:5000] # LIMITE DI 5000 IMMAGINI PER EVITARE PROBLEMI DI MEMORIA

# <span style="color:orange; font-size: 1em;"> Funzioni

In [ ]:
def calculate_2dft(input):
    ft = np.fft.ifftshift(input)
    ft = np.fft.fft2(ft)
    return np.fft.fftshift(ft)

######################################################################################################################
######################################################################################################################

def butterworth_2d_bandpass(image, lowcut, highcut, order=5):
    ny, nx = image.shape
    y, x = np.ogrid[:ny, :nx]
    cy, cx = ny // 2, nx // 2
    # Create normalized radius grid (distance from center)
    radius = np.sqrt((x - cx)**2 + (y - cy)**2)
    radius /= np.max(radius)  # Normalize to [0, 1]
    # Normalize lowcut/highcut to [0, 1] as fraction of Nyquist
    low = lowcut * 2
    high = highcut * 2
    # Butterworth bandpass formula
    def butterworth(freq, cutoff, n):
        return 1 / (1 + (freq / cutoff)**(2 * n))
    # Bandpass = Highpass * Lowpass
    lowpass = butterworth(radius, high, order)
    highpass = 1 - butterworth(radius, low, order)
    bandpass_mask = lowpass * highpass
    # Apply filter in frequency domain
    fft_image = np.fft.fft2(image)
    fft_shifted = np.fft.fftshift(fft_image)
    filtered_fft = fft_shifted * bandpass_mask
    filtered_image = np.fft.ifft2(np.fft.ifftshift(filtered_fft)).real
    return filtered_image, bandpass_mask


######################################################################################################################
######################################################################################################################

def crop_center(image, size):
    """
    Crop a square region of given size around the image center.

    """
    ny, nx = image.shape
    cx, cy = nx // 2, ny // 2
    half = size // 2
    x_min = cx - half
    x_max = cx + half
    y_min = cy - half
    y_max = cy + half
    return image[y_min:y_max, x_min:x_max]

######################################################################################################################
######################################################################################################################

def plot_psd_peak(freqs, ps1D, largest_peak_freq, frequency_range, expected_speckle_size, plate_scale):
    zoom_range = (largest_peak_freq - frequency_range / 2, largest_peak_freq + frequency_range / 2)
    zoom_indices = (freqs >= zoom_range[0]) & (freqs <= zoom_range[1])
    zoom_freqs = freqs[zoom_indices]
    zoom_power = ps1D[zoom_indices]
    plt.figure(figsize=(8, 6))
    plt.loglog(zoom_freqs, zoom_power, label='Zoomed Power Spectrum')  # Emphasize the zoomed region
    plt.axvline(x=largest_peak_freq, color='red', linestyle='--', label=f'Peak: {largest_peak_freq:.4f} pix⁻¹')
    plt.axvline(x=1/(expected_speckle_size/plate_scale), color='magenta', linestyle='--', label='Expected Speckle Size')
    plt.title("Zoomed View of Power Spectrum near Largest Peak")
    plt.xlabel("Spatial Frequency [pix⁻¹]")
    plt.ylabel("Power")
    plt.title("Zoomed View of Power Spectrum near Largest Peak")
    plt.grid(True, which="both", linestyle='--')
    plt.legend()
    plt.show()
    
##################################################################################
######################################################################################
def find_peak_power(freqs, ps1D, expected_speckle_size_pixels, frequency_range):
    """
    frequency_range : float
        Range around the expected speckle frequency to search for the peak.

    largest_peak_freq : float
        Frequency with the largest power within the specified range.
    largest_peak_power : float
        Largest power within the specified frequency range.
    """
    expected_speckle_frequency = 1 / expected_speckle_size_pixels
    lower_freq = expected_speckle_frequency - frequency_range / 2
    upper_freq = expected_speckle_frequency + frequency_range / 2
    #print(expected_speckle_size_pixels,expected_speckle_frequency, lower_freq, upper_freq)
    peak_indices = (freqs >= lower_freq) & (freqs <= upper_freq)

    if not any(peak_indices):
        print("No frequencies selected within the specified range.")
        return None, None

    selected_freqs = freqs[peak_indices]
    selected_power = ps1D[peak_indices]
    largest_peak_index = np.argmax(selected_power)
    largest_peak_freq = selected_freqs[largest_peak_index]
    largest_peak_power = selected_power[largest_peak_index]
    
    return largest_peak_freq, largest_peak_power

# <span style="color:red; font-size: 1em;"> Codice

In [ ]:

header = Header()
header['CDELT1'] = 1.0
header['CDELT2'] = 1.0

peak_frequencies = []
peak_powers = []

for i, image in enumerate(immagini):
    
    largest_peak_freq = None
    largest_peak_power = None
    
    if stacked and i + 3 <= len(immagini):
        data = np.sum(immagini[i:i + 3], axis=0)
    else:
        data = image
    image_cropped = crop_center(data, crop_size)
    data = image_cropped.copy()
    background_level = np.median(data)
    data = data - background_level
    data[data < 0] = 0
    image = data.copy()

    # Apply 2D Butterworth bandpass
    filtered_image, mask = butterworth_2d_bandpass(image, lowcut, highcut, ordine)
    # Normalize filtered image
    filtered_image = np.nan_to_num(filtered_image, nan=0.0, posinf=0.0, neginf=0.0)
    filtered_image[filtered_image < 0] = 0
    filtered_image /= (np.max(filtered_image) + 1e-8)

    # Compute power spectrum
    pspec_filtered = PowerSpectrum(filtered_image, header=header, distance=1 * u.pc)
    pspec_filtered.run(verbose=False, xunit=u.pix**-1)
    
    freqs = pspec_filtered.freqs.value
    ps1D = pspec_filtered.ps1D
    largest_peak_freq, largest_peak_power = find_peak_power(freqs, ps1D, expected_speckle_size_pixels, frequency_range)

    if largest_peak_freq is not None and largest_peak_power is not None:
        print(f"Largest peak frequency: {largest_peak_freq:.4f} pix⁻¹")
        print(f"Largest peak power: {largest_peak_power:.2f}")
        #print(f"Expected speckle size: {expected_speckle_size:.4f} arcsec")
        #print(f"Expected speckle size in pixels: {expected_speckle_size_pixels:.4f} pixels")
        #plot_psd_peak(freqs, ps1D, largest_peak_freq, frequency_range, expected_speckle_size, plate_scale)
    else:
        print("Could not find any peaks within the specified frequency range.")

    peak_frequencies.append(largest_peak_freq)
    peak_powers.append(largest_peak_power)
  


# <span style="color:blue; font-size: 1em;"> Salvataggio in file

In [ ]:



output_folder = 'outputs'
os.makedirs(output_folder, exist_ok=True)
output_csv_file = os.path.join(output_folder, 'peak_frequencies.csv')
output_fits_file = os.path.join(output_folder, 'peak_frequencies.fits')
with open(output_csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Peak Frequencies', 'Peak Power'])
    for freq, power in zip(peak_frequencies, peak_powers):
        writer.writerow([freq, power])
print(f"Data saved to {output_csv_file}")
hdu = fits.PrimaryHDU(data=np.array([peak_frequencies, peak_powers]))
hdul = fits.HDUList([hdu])
hdul.writeto(output_fits_file, overwrite=True)
print(f"Data saved to {output_fits_file}")